In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%matplotlib inline
import time
from typing import List, Dict
import random
import os
import torch
import cv2
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from tqdm import tqdm
from PIL import Image
from facenet_pytorch import MTCNN

In [ ]:
torch.__version__

'2.2.1+cu121'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
def cropface(img, x, y, w, h):
    height, width, chan = img.shape
    xmn = max(0,int(x))
    xmx = min(width,int(x+w))
    ymn = max(0,int(y))
    ymx = min(height,int(y+h))
    crop_img = img[ymn:ymx, xmn:xmx]
    return crop_img

In [ ]:
#face detector is from https://github.com/keyurr2/face-detection

modelFile = '/content/drive/MyDrive/minichallenge/res10_300x300_ssd_iter_140000.caffemodel'
configFile = '/content/drive/MyDrive/minichallenge/deploy.prototxt.txt'
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

In [ ]:
#get the highest confidence face detected
def getface(image,net):

    h, w = image.shape[:2]

    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,(300, 300), (104.0, 117.0, 123.0))
    net.setInput(blob)
    faces = net.forward()
    arr_face = []
    cont = 0
    for i in range(faces.shape[2]):
        confidence = faces[0,0,i,2]
        if confidence > 0.4:
            cont += 1
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            (x, y, x1, y1) = (x - 10, y - 10, x1+10, y1+10)
            imgcrop = cropface(image,x,y,abs(x1-x),abs(y1-y))
            arr_face.append(imgcrop)
        if cont > 0:
            break

    return arr_face


In [ ]:
def train(model: nn.Module,
          loss_fn: nn.modules.loss._Loss,
          optimizer: torch.optim.Optimizer,
          train_loader: torch.utils.data.DataLoader,
          epoch: int=0)-> List:
    train_loss = []
    model.train()
    for batch_idx,sample_batch in enumerate(tqdm(train_loader)):
      # Zero the parameter gradients
      optimizer.zero_grad()
      images = sample_batch['image']
      targets = sample_batch['target']
      images = images.float()
      images = images.to(device)
      targets = targets.to(device)
      # Forward + backward + optimize
      output = model(images)
      loss = loss_fn(output, targets)
      loss.backward()
      optimizer.step()


      train_loss.append(loss.item())
    assert len(train_loss) == len(train_loader)
    return train_loss


In [ ]:
def test(model: nn.Module,
         loss_fn: nn.modules.loss._Loss,
         test_loader: torch.utils.data.DataLoader,
         epoch: int=0)-> Dict:
    test_loss = 0
    correct = 0
    total_num = 0
    model.eval()
    prediction = torch.empty(0).to(device)
    accbatch = []
    with torch.no_grad():
      for batch_idx,sample_batch in enumerate(test_loader):
        images = sample_batch['image']
        targets = sample_batch['target']
        images = images.to(device)
        images = images.float()
        targets = targets.to(device)
        output = model(images)
        test_loss += loss_fn(output, targets).item()
        pred = output.data.max(1, keepdim=True)[1]

        pred = torch.flatten(pred, 0)
        prediction = torch.cat((prediction,pred),dim=0)

        correct += pred.eq(targets.data.view_as(pred)).sum()
        total_num = total_num+1

    test_loss /= len(test_loader)
    accuracy = (correct/len(test_loader.dataset))

    test_stat = {"loss":test_loss,"accuracy":accuracy,"prediction":prediction}

    print(f"Test result on epoch {epoch}: total sample: {total_num}, Avg loss: {test_stat['loss']:.3f}, Acc: {100*test_stat['accuracy']:.3f}%")

    return test_stat

In [ ]:
#set all seeds to 0
def set_seed(seed: int = 0) -> None:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
set_seed(0)

In [ ]:
pathCsv = '/content/drive/MyDrive/minichallenge/train.csv'
pathCat = '/content/drive/MyDrive/minichallenge/category.csv'
celebcsv = pd.read_csv(pathCsv, header = 0)
celebcsv.set_index('Category',inplace=True)
celebcat = pd.read_csv(pathCat,header = 0)
celebcat.set_index('Category',inplace=True)


In [ ]:
celebcsv = pd.merge(celebcsv,celebcat,left_index=True, right_index=True)
celebcsv = celebcsv.drop('Unnamed: 0_x',axis=1)

In [ ]:
celebdata = celebcsv.to_numpy()

In [ ]:
np.random.shuffle(celebdata)

In [ ]:
pathFace = '/content/drive/MyDrive/minichallenge/train'

In [ ]:
listitems = os.listdir(pathFace)

In [ ]:
print(len(listitems))

69630


In [ ]:
pathFace = '/content/drive/MyDrive/minichallenge/train'
pathsave = '/content/drive/MyDrive/minichallenge/faces'
new_arr = []
cont = 0
for idx,name in enumerate(tqdm(celebdata)):
    image_pil = Image.open(str(os.path.join(pathFace, name[0]))).convert('RGB')
    image = np.array(image_pil)
    image = image[:, :, ::-1].copy()

    faces = getface(image,net)
    for face in faces:
        cv2.imwrite(str(os.path.join(pathsave, str(cont)+'.jpg')),face)
        new_arr.append([str(cont)+'.jpg',name[1]])
        cont = cont+1


celebndata_train = np.array(new_arr,dtype="object")

In [ ]:
transformdata = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Resize((224,224)),
                                                torchvision.transforms.RandomHorizontalFlip(),
                                               torchvision.transforms.RandomRotation(30),

                                               torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                                               ])

transformtest = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Resize((224,224)),
                                                torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [ ]:
class CelebrityDataset(Dataset):
  def __init__(self, dataset, pathFace, transforms = None):
    self.celebdframe = dataset
    self.pathFace = pathFace
    self.transform = transforms

  def __len__(self):
    return len(self.celebdframe)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
            idx = idx.tolist()

    imgName = os.path.join(self.pathFace, self.celebdframe[idx,0])
    image = cv2.imread(imgName,cv2.IMREAD_GRAYSCALE)
    image = cv2.cvtColor(image,cv2.COLOR_GRAY2BGR)
    label = int(self.celebdframe[idx, 1])

    tlabel = torch.tensor(label)

    if self.transform:
      timage = self.transform(image)
    sample = {'image': timage, 'target': tlabel}


    return sample

In [ ]:
np.random.shuffle(celebndata)
pathFace = '/content/drive/MyDrive/minichallenge/faces'
traindata = celebndata[:int(len(celebndata)*0.8)]
testdata = celebndata[int(len(celebndata)*0.8):]
celebdataset = CelebrityDataset(traindata,pathFace,transformdata)
celeb_test = CelebrityDataset(testdata,pathFace,transformtest)

In [ ]:
train_loader = torch.utils.data.DataLoader(celebdataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(celeb_test,batch_size=100, shuffle=True)

In [ ]:
import torchvision.models as models

In [ ]:
model = models.resnet34(weights='DEFAULT')
model.fc = nn.Linear(in_features=512, out_features=100)
model = model.to(device)

In [ ]:
pathd = '/content/drive/MyDrive/minichallenge/best_model.pt'

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9)

In [ ]:

set_seed(0)
max_epoch = 10
highest_acc = 0
for epoch in range(1,max_epoch+1):
  loss = train(model,criterion,optimizer,train_loader,epoch)
  set_seed()
  test_stat = test(model,criterion,test_loader,epoch)
  if test_stat['accuracy'] > highest_acc:
    highest_acc = test_stat['accuracy']
    best_model = model

torch.save(best_model,pathd)

In [ ]:
best_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
def getfacetest(image,net):
    #we increase the borders since big faces didn't work on the face detector. This wasn't done for the first part since it worked best this way
    #We also use all faces detected in the image
    h, w = image.shape[:2]
    image = cv2.copyMakeBorder(image,int(h*0.4),int(h*0.4),int(w*0.4),int(w*0.4), cv2.BORDER_CONSTANT, None,0)
    h, w = image.shape[:2]

    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,(300, 300), (104.0, 117.0, 123.0))
    net.setInput(blob)
    faces = net.forward()
    arr_face = []
    cont = 0
    #print(faces[0,0,:,2])
    for i in range(faces.shape[2]):
        confidence = faces[0,0,i,2]
        if confidence > 0.4:

            cont += 1
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            width = x1 - x
            height = y1 - y
            (x, y, x1, y1) = (x - int(width*0.10), y - int(height*0.10), x1+int(width*0.10), y1+int(height*0.10))
            imgcrop = cropface(image,x,y,abs(x1-x),abs(y1-y))
            arr_face.append(imgcrop)


    return arr_face

In [ ]:
pathFace = '/content/drive/MyDrive/minichallenge/test'
new_arr = []
t_arr = []
cont = 0
for idx,name in enumerate(tqdm(os.listdir(pathFace))):
    image_pil = Image.open(str(os.path.join(pathFace, name))).convert('RGB')
    image = np.array(image_pil)
    image = image[:, :, ::-1].copy()
    nname, _ = name.split('.')
    nname = int(nname)
    arr_faces = []

    faces = getfacetest(image,net)
    for face in faces:
        try:
            facecrop = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            facecrop = cv2.cvtColor(facecrop,cv2.COLOR_GRAY2BGR)
            arr_faces.append(facecrop)
        except:
            pass

    new_arr.append([nname,arr_faces])

celebndata = np.array(new_arr,dtype="object")

In [ ]:
nmodel = best_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
nmodel.eval().to(device)
res = []
for idx,i in enumerate(tqdm(celebndata)):
    max_prob = 0
    class_pred = 0
    for face in i[1]:
        inputs = transformtest(face)
        inputs = inputs.to(device,dtype=torch.float)
        inputs = torch.unsqueeze(inputs,0)
        output = nmodel(inputs)
        class_prob = output.data.max(1, keepdim=True)[1].flatten().tolist()[0]
        pred = output.data.max(1, keepdim=True)[1]
        pred = torch.flatten(pred, 0).to('cpu').tolist()[0]
        #we only keep the highest probability class for all faces detected in one image
        if class_prob > max_prob:
            max_prob = class_prob
            class_pred = pred
            break
    res.append([i[0],class_pred])

In [ ]:
dataframe1 = pd.DataFrame(res,columns = ['Id','Category'])

In [ ]:
dataframe1.astype('int32')

,Id,Category
0,0,5
1,1,21
2,10,7
3,100,71
4,1000,23
...,...,...
4972,995,82
4973,996,7
4974,997,11
4975,998,75


In [ ]:
submis = pd.merge(dataframe2,dataframe1,left_index=True, right_index=True)
submis = submis.drop('Id_y',axis=1)
submis = submis.rename(columns={'Id_x':'Id'})
submis = submis.sort_values('Id',key=pd.to_numeric)

In [ ]:
pathCat = '/content/drive/MyDrive/minichallenge/category.csv'
celebcat = pd.read_csv(pathCat,header = 0)
celebcat = celebcat.rename(columns={'Category':'Name'})
celebcat = celebcat.rename(columns={'Unnamed: 0':'Category'})

In [ ]:
submis = pd.merge(dataframe1,celebcat,on='Category')
submis = submis.sort_values('Id',key=pd.to_numeric)

In [ ]:
submis

,Id,Category,Name
0,0,5,Anna Friel
55,1,21,Amy Ryan
4860,2,52,Adrien Brody
2997,3,26,Adriana Barraza
2185,4,72,Albert Brooks
...,...,...,...
3148,4972,69,Barabara Palvin
100,4973,21,Amy Ryan
537,4974,95,Bill Hader
1942,4975,25,Amanda Crew


In [ ]:
#submis= submis.set_index('Id')
submis = submis.drop(columns=['Category'])
submis = submis.rename(columns={'Name':'Category'})
submis

,Id,Category
0,0,Anna Friel
55,1,Amy Ryan
4860,2,Adrien Brody
2997,3,Adriana Barraza
2185,4,Albert Brooks
...,...,...
3148,4972,Barabara Palvin
100,4973,Amy Ryan
537,4974,Bill Hader
1942,4975,Amanda Crew


In [ ]:
pathc = '/content/drive/MyDrive/minichallenge/submission.csv'
submis.to_csv(pathc,index = False)